# Comparing O2 Rich Atmosphere with Modern Earth (Reflected Light Spectras)

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import copy
import pandas as pd
from scipy import optimize
from matplotlib import pyplot as plt
import pickle
from itertools import cycle
import matplotlib.colors as mcolors
import astropy.units as u

from photochem.utils import stars
import PICASO_Climate_grid_121625 as picaso_grid
import Photochem_grid_121625 as photochem_grid
import Reflected_Spectra_grid_13026 as Reflected_Spectra
from picaso.photochem import EquilibriumChemistry
import GraphsKey

import os
from pathlib import Path

current_directory = Path.cwd()
references_directory_path = "Installation&Setup_Instructions/picasofiles/reference"
PYSYN_directory_path = "Installation&Setup_Instructions/picasofiles/grp/redcat/trds"
print(os.path.join(current_directory, references_directory_path))
print(os.path.join(current_directory, PYSYN_directory_path))

os.environ['picaso_refdata']= os.path.join(current_directory, references_directory_path)
os.environ['PYSYN_CDBS']= os.path.join(current_directory, PYSYN_directory_path)


import picaso.justdoit as jdi
import picaso.justplotit as jpi

/mnt/c/Users/lily/Documents/NASAUWPostbac/MiniNeptuneGrid26_PostBac/Installation&Setup_Instructions/picasofiles/reference
/mnt/c/Users/lily/Documents/NASAUWPostbac/MiniNeptuneGrid26_PostBac/Installation&Setup_Instructions/picasofiles/grp/redcat/trds


## Function that Calculates Photochem, pressure at the bottom of a cloud, and reflected spectrum of a certain input w/ Photochem Results

In [22]:
import os

def Photochem_called(rad_plan=None, log10_planet_metallicity=None, tint=None, semi_major=None, ctoO=None, log_Kzz=None, PIC_PT=None, outputfile=None):
    # Planet Parameters
    atoms_names = ['H', 'He', 'N', 'O', 'C'] # We select a subset of the atoms in zahnle_earth.yaml (leave out Cl), remove Sulpher for faster convergence

    # Calculate the Mass of the Planet and Teq
    mass_planet_earth = picaso_grid.mass_from_radius_chen_kipping_2017(R_rearth=rad_plan)
    mass_planet = mass_planet_earth * (5.972e+24) * 1e3 # of planet, but in grams
    radius_planet = rad_plan * (6.371e+6) * 1e2 # of planet but in cm
    solar_zenith_angle = 60 # Used in Tsai et. al. (2023), in degrees
    planet_Teq = picaso_grid.calc_Teq_SUN(distance_AU=semi_major)

    # Dependent constant variables
    if os.path.exists(f'sun_flux_file_{planet_Teq}'):
        stellar_flux_file = f'sun_flux_file_{planet_Teq}'
        print(f"The stellar flux file already exists")
    else:
        wv, F = star_spectrum.solar_spectrum(Teq=planet_Teq, outputfile= f'sun_flux_file_{planet_Teq}')
        stellar_flux_file = f'sun_flux_file_{planet_Teq}'

    pressure = PIC_PT['pressure']
    temperature = PIC_PT['temperature']
    PT_list = np.array([pressure, temperature]) 

    # Test Data - This works fine.
    #with open('out_Sun_5778_initp3bar.pkl', 'rb') as file:
    #    out_reopened = pickle.load(file)
    #    pressure = out_reopened['pressure']
    #    temperature = out_reopened['temperature']
    #PT_list = np.array(pressure), np.array(temperature)
    #convergence_values = np.array([1])

    # Define P-T Profile (convert from PICASO to Photochem)
    P_extended, T_extended = photochem_grid.linear_extrapolate_TP(PT_list[0], PT_list[1]) # Extend the end to bypass BOA Error of mismatching boundary conditions.
    #P = np.flip(np.array(PT_list[0]) * (10**6)).copy()
    #T = np.flip(np.array(PT_list[1])).copy()
    P = np.flip(np.array(P_extended) * (10**6)).copy() # Convert from bars to dynes/cm^2
    T = np.flip(np.array(T_extended)).copy()
    
    # Check if numpy array is sorted (investigating error)
    sorted_P = np.flip(np.sort(P)).copy()
    unsorted_indices = np.where(P != sorted_P)[0]
    
    # Generate reaction & thermodynamic files for gas giants
    photochem_grid.zahnle_rx_and_thermo_files(
    atoms_names=atoms_names,
    rxns_filename='photochem_rxns.yaml',
    thermo_filename='photochem_thermo.yaml',
    remove_reaction_particles=True # For gas giants, we should always leave out reaction particles.
    )

    # Initialize ExoAtmosphereGasGiant
    # Assigns 
    pc = photochem_grid.gasgiants.EvoAtmosphereGasGiant(
        mechanism_file='photochem_rxns.yaml',
        stellar_flux_file=stellar_flux_file,
        planet_mass=mass_planet,
        planet_radius=radius_planet,
        solar_zenith_angle=solar_zenith_angle,
        thermo_file='photochem_thermo.yaml'
    )
    # Adjust convergence parameters:
    pc.var.conv_longdy = 0.03 # converges at 3% (change of mixing ratios over long time)
    pc.gdat.max_total_step = 10000 # assumes convergence after 10,000 steps
    
    pc.gdat.verbose = True # printing
    
    # Define the host star composition
    molfracs_atoms_sun = np.ones(len(pc.gdat.gas.atoms_names))*1e-10 # This is for the Sun
    comp = {
        'H' : 9.21e-01,
        'N' : 6.23e-05,
        'O' : 4.51e-04,
        'C' : 2.48e-04,
        'S' : 1.21e-05,
        'He' : 7.84e-02
    }

    tot = sum(comp.values())
    for key in comp:
        comp[key] /= tot
    for i,atom in enumerate(pc.gdat.gas.atoms_names):
        molfracs_atoms_sun[i] = comp[atom]
    
    pc.gdat.gas.molfracs_atoms_sun = molfracs_atoms_sun

    # Assume a default radius for particles 1e-5cm was default, so we increased the size but think of these in microns
    particle_radius = pc.var.particle_radius
    particle_radius[:,:] = 1e-3 #cm or 10 microns
    pc.var.particle_radius = particle_radius

    # Assumed Kzz (cm^2/s) in Tsai et al. (2023)
    Kzz_zero_grid = np.ones(P.shape[0])
    Kzz = Kzz_zero_grid*(10**log_Kzz) #Note Kzz_fac was meant to be the power of 10 since we are in log10 space

    # Initialize the PT based on chemical equilibrium 
    pc.gdat.BOA_pressure_factor = 3
    pc.initialize_to_climate_equilibrium_PT(P, T, Kzz, 10**log10_planet_metallicity, ctoO)
    
    # Integrate to steady state
    converged = pc.find_steady_state()

    # Check if the model converged after 10,000 steps
    if not converged:
        assert pc.gdat.total_step_counter > pc.gdat.max_total_step - 10
        
    sol_raw = pc.return_atmosphere()
    soleq_raw = pc.return_atmosphere(equilibrium=True)

    # Call the interpolation of the grid 
    sol = photochem_grid.interpolate_photochem_result_to_nlayers(out=sol_raw, nlayers=100)
    soleq = photochem_grid.interpolate_photochem_result_to_nlayers(out=soleq_raw, nlayers=100)
    #convergence_values = np.array([convergence_values[0] for _ in range(len(sol['pressure']))])
    #converged = np.array([converged for _ in range(len(sol['pressure']))])

    # Print out the lengths of arrays: Save the size of the grid for future reference.

    print(f"This is for the input value of planet radius:{rad_plan}, metal:{float(log10_planet_metallicity)}, tint:{tint}, semi major:{semi_major}, ctoO: {ctoO}, log_Kzz: {log_Kzz}")
    
    #for key, value in sol.items():
    #    if isinstance(value, np.ndarray):  # Check if the value is a list (or array)
    #        print(f"The array for sol's '{key}' has a length of: {len(value)}")
    #    else:
    #        print(f"The value for sol's '{key}' is not an array.")

    #for key, value in soleq.items():
    #    if isinstance(value, np.ndarray):  # Check if the value is a list (or array)
    #        print(f"The array for soleq's '{key}' has a length of: {len(value)}, Length of pressure: {len(P)}")
    #    else:
    #        print(f"The value for soleq's '{key}' is not an array.")

    # Add nan's to fit the grid if underestimated, and make sure list goes from largest to smallest.

    if outputfile == None:
        return sol, soleq, pc

    else:
        with open(f'sol_{outputfile}.pkl', 'wb') as f:
            pickle.dump(sol, f)
        with open(f'soleq_{outputfile}.pkl', 'wb') as f:
            pickle.dump(soleq, f)
        return sol, soleq, pc
        

In [23]:
def find_pbot(sol=None, solaer=None, tol=0.9):

    """
    Parameters:
    pressures: ndarray
        Pressure at each atmospheric layer in dynes/cm^2
    H2Oaer: ndarray
        Mixing ratio of H2O aerosols.
    tol: float, optional
        The threshold value for which we define the beginning of the cloud, 
        by default 1e-4. 

    Returns:
    P_bottom: float
        The cloud bottom pressure in dynes/cm^2
        
    """

    pressure = sol['pressure']
    H2Oaer = solaer['H2Oaer']

    # There is no water cloud in the model, so we return None
    # For the cloud bottom of pressure

    if np.max(H2Oaer) < 1e-20:
        return None

    # Normalize so that max value is 1
    H2Oaer_normalized = H2Oaer/np.max(H2Oaer)

    # loop from bottom to top of atmosphere, cloud bottom pressure
    # defined as the index level where the normalized cloud mixing ratio
    # exeeds tol .

    ind = None
    
    for i, val in enumerate(H2Oaer_normalized):
        if val > tol:
            ind = i
            break

    if ind is None:
        raise Exception('A problem happened when trying to find the bottom of the cloud.')

    # Bottom of the cloud
    pbot = pressure[ind]

    return pbot

In [24]:
def reflected_spectrum_planet_Sun(rad_plan=None, planet_metal=None, tint=None, semi_major=None, ctoO=None, Kzz=None, phase_angle=None, sol_path=None, soleq_path=None, Photochem_file=None, atmosphere_kwargs={}):

    """
    This finds the reflected spectra of a planet similar to K218b around a Sun.

    Parameters:
    rad_plan: float
        This is the radius of the planet in units of Earth radii.
    planet_metal: float
        This is the planet's metallicity in units of log10 x Solar metallicity.
    tint: float
        This is the planet's internal temperature in Kelvin.
    semi_major: float
        This is the semi major axis of your planet's orbit in units of AU.
    ctoO: float
        This is the carbon to oxygen ratio of your planet's atmosphere in units of xSolar c/o.
    Kzz: float
        This is the eddy diffusion coefficient in logspace (i.e. the power of 10) in cm/s^2.
    phase_angle: float
        This is the phase of orbit the planet is in relative to its star and the observer (i.e. how illuminated it is), units of radians.
    Photochem_file: string
        This is the path to the Photochem grid you would like to pull composition information from.
    atmosphere_kwargs: dict 'exclude_mol': value where value is a string
        If left empty, all molecules are included, but can limit how many molecules are calculated. 

    Results: IDK for sure though
    wno: grid of 150 points
        ???
    fpfs: grid of 150 points
        This is the relative flux of the planet and star (fp/fs). 
    alb: grid of 150 points
        ???
    np.array(clouds): grid of 150 points
        This is a grid of whether or not a cloud was used to make the reflective spectra using the binary equivalent to booleans (True=1, False=0).
        
    """

    opacity = jdi.opannection(method='resortrebin', wave_range=[0.3,2.5])

    planet_metal = float(planet_metal)
    
    start_case = jdi.inputs()

    # Then calculate the composition from the TP profile
    class planet:
        
        planet_radius = (rad_plan*6.371e+6*u.m) # in meters
        planet_mass = picaso_grid.mass_from_radius_chen_kipping_2017(R_rearth=rad_plan)*(5.972e+24) # in kg
        planet_Teq = picaso_grid.calc_Teq_SUN(distance_AU=semi_major) # Equilibrium temp (K)
        planet_grav = (const.G * (planet_mass)) / ((planet_radius)**2) # of K2-18b in m/s^2
        planet_ctoO = ctoO # in xSolar

    class Sun:
        
        stellar_radius = 1 # Solar radii
        stellar_Teff = 5778 # K
        stellar_metal = 0.0 # log10(metallicity)
        stellar_logg = 4.4 # log10(gravity), in cgs units

    solar_zenith_angle = 60 # Used in Tsai et al. (2023)
        
    # Star and Planet Parameters (Stay the Same & Should Match Photochem & PICASO)
    start_case.phase_angle(phase_angle, num_tangle=8, num_gangle=8) #radians, using default here

    jupiter_mass = const.M_jup.value # in kg
    jupiter_radius = 69911e+3 # in m
    start_case.gravity(gravity=planet.planet_grav, gravity_unit=jdi.u.Unit('m/(s**2)'), radius=(planet.planet_radius.value)/jupiter_radius, radius_unit=jdi.u.Unit('R_jup'), mass=(planet.planet_mass.value)/jupiter_mass, mass_unit=jdi.u.Unit('M_jup'))
    
    # star temperature, metallicity, gravity, and opacity (default opacity is opacity.db in the reference folder)
    start_case.star(opannection=opacity, temp=Sun.stellar_Teff, logg=Sun.stellar_logg, semi_major=semi_major, metal=Sun.stellar_metal, radius=Sun.stellar_radius, radius_unit=jdi.u.R_sun, semi_major_unit=jdi.u.au)

    # Match Photochemical Files
    if Photochem_file is not None:
        sol_dict, soleq_dict, PT_list, convergence_PC, convergence_TP = Reflected_Spectra.find_Photochem_match(filename=Photochem_file, rad_plan=rad_plan, log10_planet_metallicity=planet_metal, tint=tint, semi_major=semi_major, ctoO=ctoO,Kzz=Kzz)

    else:
        with open(sol_path, 'rb') as file:
            sol = pickle.load(file)
        with open(soleq_path, 'rb') as file:
            soleq = pickle.load(file)
            
    # Determine Planet Atmosphere & Composition

    atm, sol_dict_aer = Reflected_Spectra.make_picaso_atm(sol_dict) # Converted Pressure of Photochem, in dynes/cm^2, back to bars and flip all arrays before placing into PICASO
    df_atmo = jdi.pd.DataFrame(atm)

    if 'exclude_mol' in atmosphere_kwargs:
        sp = atmosphere_kwargs['exclude_mol'][0]
        if sp in df_atmo:
            df_atmo[sp] *= 0
            
    start_case.atmosphere(df = df_atmo) 
    df_cldfree = start_case.spectrum(opacity, calculation='reflected', full_output=True)
    wno_cldfree, alb_cldfree, fpfs_cldfree = df_cldfree['wavenumber'], df_cldfree['albedo'], df_cldfree['fpfs_reflected']
    _, alb_cldfree_grid = jdi.mean_regrid(wno_cldfree, alb_cldfree, R=150)
    wno_cldfree_grid, fpfs_cldfree_grid = jdi.mean_regrid(wno_cldfree, fpfs_cldfree, R=150)

    print(f'This is the length of the grids created: {len(wno_cldfree_grid)}, {len(fpfs_cldfree_grid)}')

    # Determine Whether to Add Clouds or Not?

    if "H2Oaer" in sol_dict_aer:
        # What if we added Grey Earth-like Clouds?
        
        # Calculate pbot:
        pbot = find_pbot(sol = atm, solaer=sol_dict_aer)

        if pbot is not None:
            print(f'pbot was calculated, there is H2Oaer and a cloud was implemented')
            logpbot = np.log10(pbot)
        
            # Calculate logdp:
            ptop_earth = 0.6
            pbot_earth = 0.7
            logdp = np.log10(pbot_earth) - np.log10(ptop_earth)  
    
            # Default opd (optical depth), w0 (single scattering albedo), g0 (asymmetry parameter)
            start_case.clouds(w0=[0.99], g0=[0.85], 
                              p = [logpbot], dp = [logdp], opd=[10])
            # Cloud spectrum
            df_cld = start_case.spectrum(opacity,full_output=True)
            
            # Average the two spectra - This differs between Calculating Earth Reflected Spectra 
            wno_c, alb_c, fpfs_c, albedo_c = df_cld['wavenumber'],df_cld['albedo'],df_cld['fpfs_reflected'], df_cld['albedo']
            _, alb = jdi.mean_regrid(wno_cldfree, 0.5*alb_cldfree+0.5*albedo_c,R=150)
            wno, fpfs = jdi.mean_regrid(wno_cldfree, 0.5*fpfs_cldfree+0.5*fpfs_c,R=150)

            # Match the length of the clouds array with the length of wno or alb (fpfs is different length)
            clouds = [1] * len(wno)

            return wno, fpfs, alb, np.array(clouds)

        else:
            print(f'pbot is empty, so no cloud is implemented')
            wno = wno_cldfree_grid.copy()
            alb = alb_cldfree_grid.copy()
            fpfs = fpfs_cldfree_grid.copy()

            # Match the length of the clouds array with the length of wno or alb (fpfs is different length)
            clouds = [0] * len(wno)

            print(f'This is the length of the values I want to save: wno {len(wno)}, alb {len(alb)}, fpfs {len(fpfs)}, clouds {len(clouds)}')

            return wno, fpfs, alb, np.array(clouds)

    else:
        print(f'H2Oaer is not in solutions')
        wno = wno_cldfree_grid.copy()
        alb = alb_cldfree_grid.copy()
        fpfs = fpfs_cldfree_grid.copy()
        print(f'For the inputs: {rad_plan}, {planet_metal}, {tint}, {semi_major}, {ctoO}, {Kzz}, {phase_angle}, The length should match: wno - {len(wno)}, alb - {len(alb)}, fpfs - {len(fpfs)}')
        
        # Match the length of the clouds array with the length of wno or alb (fpfs is different length)
        clouds = [0] * len(wno) # This means that there are no clouds

        return wno, fpfs, alb, np.array(clouds)

### Example Application of O2 Rich K218b
Parameters:
- planet radius: 2.61x Earth
- **metallicity: 3.5x (logspace) x solar (~3000x solar metallicity)**
- tint: 155K
- semi major in AU: 1 AU
- **ctoO_solar: 0.01 x solar c/o ratio**

In [12]:
file_path = "/mnt/c/Users/lily/Documents/NASAUWPostbac/MiniNeptuneGrid26_PostBac/out_3.5m_0.01co.pkl"
with open(file_path, 'rb') as file:
    data_35m_001co = pickle.load(file)
    print(data_35m_001co)

{'pressure': array([1.00000000e-06, 1.25892541e-06, 1.58489319e-06, 1.99526231e-06,
       2.51188643e-06, 3.16227766e-06, 3.98107171e-06, 5.01187234e-06,
       6.30957344e-06, 7.94328235e-06, 1.00000000e-05, 1.25892541e-05,
       1.58489319e-05, 1.99526231e-05, 2.51188643e-05, 3.16227766e-05,
       3.98107171e-05, 5.01187234e-05, 6.30957344e-05, 7.94328235e-05,
       1.00000000e-04, 1.25892541e-04, 1.58489319e-04, 1.99526231e-04,
       2.51188643e-04, 3.16227766e-04, 3.98107171e-04, 5.01187234e-04,
       6.30957344e-04, 7.94328235e-04, 1.00000000e-03, 1.25892541e-03,
       1.58489319e-03, 1.99526231e-03, 2.51188643e-03, 3.16227766e-03,
       3.98107171e-03, 5.01187234e-03, 6.30957344e-03, 7.94328235e-03,
       1.00000000e-02, 1.25892541e-02, 1.58489319e-02, 1.99526231e-02,
       2.51188643e-02, 3.16227766e-02, 3.98107171e-02, 5.01187234e-02,
       6.30957344e-02, 7.94328235e-02, 1.00000000e-01, 1.25892541e-01,
       1.58489319e-01, 1.99526231e-01, 2.51188643e-01, 3.1622776

In [16]:
rad_plan = 2.61
log10_metal = 3.5
tint = 155
semi_major = 1
ctoO = 0.01
Kzz = 5


sol, soleq, pc = Photochem_called(rad_plan=rad_plan, log10_planet_metallicity=log10_metal, tint=tint, semi_major=semi_major, ctoO=ctoO, log_Kzz=Kzz, PIC_PT=data_35m_001co, outputfile=f'{rad_plan}_{log10_metal}_{tint}_{semi_major}_{ctoO}_{Kzz}')

The stellar flux file already exists
nsteps = 100  longdy = 9.3e-01  max_dT = 3.0e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 4.7e-07
nsteps = 200  longdy = 9.5e-01  max_dT = 3.0e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 4.7e-07
nsteps = 300  longdy = 1.7e+00  max_dT = 3.0e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 4.8e-07
nsteps = 400  longdy = 3.4e+00  max_dT = 2.9e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 4.8e-07
nsteps = 500  longdy = 1.2e+00  max_dT = 2.9e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 4.8e-07
nsteps = 600  longdy = 1.9e+00  max_dT = 2.9e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 4.8e-07
nsteps = 700  longdy = 2.3e+00  max_dT = 2.7e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 5.0e-07
nsteps = 800  longdy = 2.5e+00  max_dT = 2.8e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 5.0e-07
nsteps = 900  longdy = 3.0e+00  max_dT = 2.8e+01  max_dlog10edd = 0.0e+00  TOA_pressure = 5.0e-07
nsteps = 1000  longdy = 9.2e+00  max_dT = 2.8e+01  max_dlog10edd = 0.0e+00  TOA_p

In [ ]:
rad_plan = 2.61
log10_metal = 3.5
tint = 155
semi_major = 1
ctoO = 0.01
Kzz = 5
sol_path = f'sol_{rad_plan}_{log10_metal}_{tint}_{semi_major}_{ctoO}_{Kzz}.pkl'
soleq_path = f'soleq_{rad_plan}_{log10_metal}_{tint}_{semi_major}_{ctoO}_{Kzz}.pkl'
phase_angle = 0

wno, fpfs, alb, clouds = reflected_spectrum_planet_Sun(rad_plan=rad_plan, planet_metal=log10_metal, tint=tint, semi_major=semi_major, ctoO=ctoO, Kzz=Kzz, phase_angle=phase_angle, sol_path=sol_path, soleq_path=soleq_path, Photochem_file=None)

In [17]:
# Plot the Reflected light spectra

In [18]:
# Load up the earth spectra's
with open('archean_earth_res.pkl', 'rb') as file:
    res_archean_earth = pickle.load(file)

res_archean_earth

with open('modern_earth_res.pkl', 'rb') as file:
    res_modern_earth = pickle.load(file)

res_modern_earth

wv_archean = res_archean_earth['all'][0]
fpfs_archean = res_archean_earth['all'][1]
alb_archean = res_archean_earth['all'][2]
wv_modern = res_modern_earth['all'][0]
fpfs_modern = res_modern_earth['all'][1]
alb_modern = res_modern_earth['all'][2]

In [ ]:
wno_earth = wv_archean
fpfs_earth = fpfs_archean
lbedo_earth = alb_archean

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
ax1.plot(1e4/wno, fpfs, c='k', lw=1.5, label='K218b/G-Star')
ax1.plot(1e4/wno_earth, fpfs_earth, c='red', lw=1.5, label='Earth/G-Star')
ax1.set_xlim(0.3,1)
ax1.set_ylim(0e-10,10e-10)
ax1.set_ylabel('Planet-to-star flux ratio')
ax1.set_xlabel('Wavelength (microns)')
ax1.set_title('Reflected Spectra w/ Earth Clouds')
ax1.legend()

ax2.plot(1e4/wno, fpfs, c='k', lw=1.5, label='K218b/G-Star')
ax2.plot(1e4/wno_earth, fpfs_earth, c='red', lw=1.5, label='Earth/G-Star')
ax2.set_xlim(0.3,1)
ax2.set_ylim(0e-10,10e-9)
ax2.set_ylabel('Planet-to-star flux ratio')
ax2.set_xlabel('Wavelength (microns)')
ax2.set_title('Reflected Spectra w/ Earth Clouds')
ax2.legend()

#plt.savefig('Earth_K218b_clouds_RS.pdf',bbox_inches='tight')

#plt.savefig('Earth_K218b_clouds_RS_zoomedin.pdf',bbox_inches='tight')
plt.show()

#### A saved list of Earth spectra that changes based on phase

In [19]:
wv_archean_list = []    
fpfs_archean_list = []
wv_modern_list = []
fpfs_modern_list = []

# This is for different phases of earth
with open('earth_diff_phases.pkl', 'rb') as file:
    earth_dict = pickle.load(file)
    for phase in phase_earth:
        for key in list(earth_dict.keys()):
            if key.endswith(f'_{phase}'):
                if key.startswith(f'Archean_wv_'):
                    wv_archean = earth_dict[key]
                    wv_archean_list.append(wv_archean)
                if key.startswith(f'Archean_fpfs_'):
                    fpfs_archean = earth_dict[key]
                    fpfs_archean_list.append(fpfs_archean)
                if key.startswith(f'Modern_wv_'):
                    wv_modern = earth_dict[key]
                    wv_modern_list.append(wv_modern)
                if key.startswith(f'Modern_fpfs_'):
                    fpfs_modern = earth_dict[key]
                    fpfs_modern_list.append(fpfs_modern)

NameError: name 'phase_earth' is not defined